<a href="https://colab.research.google.com/github/thebotnets/devops/blob/master/InSwapper_Colab_FaceSwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# InSwapper Face Swap - Colab Ready
Upload your images: `man1.jpeg`, `man2.jpeg`, and `mans1.jpeg`.
The script will perform face swapping using `inswapper_128.onnx`.

In [1]:
# Install dependencies
!pip install onnxruntime
!git clone https://github.com/haofanwang/inswapper.git
%cd inswapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00
Cloning into 'inswapper'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 127 (delta 35), reused 27 (delta 26), pack-reused 83 (from 1)
Receiving objects: 100% (127/127), 784.82 KiB | 17.44 MiB/s, done.
Resolving deltas: 100% (62/62), done.
/content/inswapper


In [2]:
# Download ONNX model
!pip install insightface
!mkdir -p checkpoints
!wget -O ./checkpoints/inswapper_128.onnx "https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.3 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1060443 sha256=7fbd2cc1450cab17dfcfab6f14035306ffaa4332b87967ac604c6491cc91ee4e
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
--2025-07-26 14:43:25--  https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.103, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cas-bridge.xethub.hf.co/xet-bridge-us/64a4ee92f

In [3]:
# Upload your own images (man1.jpeg, man2.jpeg, mans1.jpeg)
from google.colab import files
uploaded = files.upload()

Saving man1.jpeg to man1.jpeg


In [8]:
# Run the swapper
from PIL import Image
from swapper import process, get_many_faces, getFaceAnalyser
import os
import cv2
import numpy as np
import onnxruntime # Import onnxruntime for providers


source_img_paths = ["man1.jpeg", "man2.jpeg"]
target_img_path = "mans1.jpeg"
# model_path = "../checkpoints/inswapper_128.onnx" # Original relative path
model_path = "/content/checkpoints/inswapper_128.onnx" # Use absolute path


# Load images
# Wrap the first source image in a list as a workaround for a potential issue in swapper.py
source_img_pil = [Image.open(source_img_paths[0])] # Renamed for clarity
target_img_pil = Image.open(target_img_path) # Renamed for clarity


# --- Face Detection Check (from cell 3391fe7d) ---
# Convert PIL images to numpy arrays in BGR format for face analysis
source_images_cv2 = [cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR) for img in source_img_pil]
target_image_cv2 = cv2.cvtColor(np.array(target_img_pil), cv2.COLOR_BGR2RGB) # Convert target image to RGB for face analysis


# Initialize face analyser
providers = onnxruntime.get_available_providers()
# Call getFaceAnalyser with model_path and providers. Let det_size use its default value.
face_analyser = getFaceAnalyser(model_path, providers)


source_faces_detected = []
for img_cv2 in source_images_cv2:
    faces = get_many_faces(face_analyser, img_cv2)
    if faces:
        source_faces_detected.extend(faces)

target_faces_detected = get_many_faces(face_analyser, target_image_cv2)

print("Detected source faces:", len(source_faces_detected))
print("Detected target faces:", len(target_faces_detected) if target_faces_detected else 0)
# --- End Face Detection Check ---


# Change source_index and target_index to '0' to swap the first detected faces (passed as strings)
try:
    # Use the original PIL images for the process function
    result_image = process(source_img_pil, target_img_pil, '0', '0', model_path) # Pass model_path

    result_image.save("result.png")
    print("Face swap completed. Result saved as result.png")

except IndexError:
    print("Error: No faces detected in the source or target image(s) by the process function. Please ensure the images contain clear faces.")
except Exception as e:
    print(f"An unexpected error occurred during face swapping: {e}")

Applied providers: ['AzureExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'AzureExecutionProvider': {}}
find model: ./checkpoints/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['AzureExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'AzureExecutionProvider': {}}
find model: ./checkpoints/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['AzureExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'AzureExecutionProvider': {}}
find model: ./checkpoints/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['AzureExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'AzureExecutionProvider': {}}
find model: ./checkpoints/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['AzureExec

In [ ]:
# Install basicsr for face restoration and enhancement
!pip install basicsr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_

In [ ]:
# Uninstall current basicsr and install a potentially compatible version
!pip uninstall basicsr -y
!pip install basicsr==1.3.4

Found existing installation: basicsr 1.4.2
Uninstalling basicsr-1.4.2:
  Successfully uninstalled basicsr-1.4.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for basicsr: filename=basicsr-1.3.4.0-py3-none-any.whl size=171810 sha256=a1973976d67582abef0565ca5a39ec217032e2121c5706c59e2c9645b83a96a5
  Stored in directory: /root/.cache/pip/wheels/04/64/cb/c981d544da20cd45bb52079046e8d94710286049194ac9ff1e
Successfully built basicsr


In [ ]:
# Clone the CodeFormer repository to get potential checkpoints or download scripts
!git clone https://github.com/sczhou/CodeFormer.git

Cloning into 'CodeFormer'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 614 (delta 209), reused 183 (delta 183), pack-reused 316 (from 2)
Receiving objects: 100% (614/614), 17.31 MiB | 33.01 MiB/s, done.
Resolving deltas: 100% (297/297), done.


In [ ]:
# List files in the expected checkpoint directory
!ls CodeFormer/CodeFormer/weights/CodeFormer/

ls: cannot access 'CodeFormer/CodeFormer/weights/CodeFormer/': No such file or directory


In [ ]:
# Create the checkpoint directory and download the CodeFormer checkpoint
!mkdir -p CodeFormer/CodeFormer/weights/CodeFormer/
!wget -O CodeFormer/CodeFormer/weights/CodeFormer/codeformer.pth 'https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth'

--2025-07-26 12:04:04--  https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/505667511/a1f9f85b-f048-428c-b18b-b79b2665a325?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-26T12%3A56%3A04Z&rscd=attachment%3B+filename%3Dcodeformer.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-26T11%3A55%3A39Z&ske=2025-07-26T12%3A56%3A04Z&sks=b&skv=2018-11-09&sig=QaKyFP4zPYCroEBltLpdip5DB8gPDknl91%2B79qVyBeI%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1MzUzMTc0NCwibmJmIjoxNzUzNTMxNDQ0LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9i

In [6]:
# Run swapper.py with enhancement options from the command line
!python swapper.py \
--source_img="./man1.jpeg" \
--target_img "./mans1.jpeg" \
--model "/content/checkpoints/inswapper_128.onnx" \
--face_restore \
--background_enhance \
--face_upsample \
--upscale=2 \
--codeformer_fidelity=0.5

usage: swapper.py [-h] --source_img SOURCE_IMG --target_img TARGET_IMG
                  [--output_img OUTPUT_IMG] [--source_indexes SOURCE_INDEXES]
                  [--target_indexes TARGET_INDEXES] [--face_restore]
                  [--background_enhance] [--face_upsample] [--upscale UPSCALE]
                  [--codeformer_fidelity CODEFORMER_FIDELITY]
swapper.py: error: unrecognized arguments: --model /content/checkpoints/inswapper_128.onnx


In [ ]:
# Remove the problematic import from swapper.py
!sed -i '/from restoration import \*/d' /content/inswapper/swapper.py

In [ ]:
# Remove the face restoration and enhancement logic from swapper.py
!sed -i '/check_ckpts()/,+40d' /content/inswapper/swapper.py # Adjust the number 40 based on the actual number of lines in the enhancement block

In [ ]:
# Remove lines causing indentation error in swapper.py
# This is an attempt to fix indentation after removing the enhancement block
!sed -i '230,+5d' /content/inswapper/swapper.py # Adjust the number 5 based on needed lines to remove